In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import pandas as pd
import time
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import os
from functools import partial


Device set: cuda


In [ ]:
#hyperparameters

bs =20
#nb_epochs = 3
num_of_images = 60
input_size = 784
hidden_sizes = [50, 50]
output_size = 10
lr = 0.01
#lr=0.003
#nb_epochs = 15
nb_epochs = 200
#lr = 0.25
#bs = 128

In [ ]:
train = datasets.MNIST(path_to_download, download=True, train=True, transform=transform)
validation = datasets.MNIST(path_to_download, download=True, train=False, transform=transform)
load_train = torch.utils.data.DataLoader(train, batch_size=20000, shuffle=True)
load_val = torch.utils.data.DataLoader(validation, batch_size=1000, shuffle=True)

In [ ]:
class new_net(nn.Module):
  def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
    super(new_net, self).__init__()

    self.layer1 = nn.Linear(input_size, hidden_size1, bias = True)
    self.layer2 =nn.Linear(hidden_size1, hidden_size2, bias =True)
    #self.layer3 = nn.Linear(hidden_size2, output_size, bias = True)

  def forward(self, x):
    y = self.layer1(x)
    y_hat = torch.relu(y)
    z = self.layer2(y)
    #z_hat = torch.relu(z)
    #scores = self.layer3(z_hat)

    return z



In [ ]:
net1=new_net(input_size=input_size,hidden_size1=hidden_sizes[0],hidden_size2=hidden_sizes[1], output_size=output_size)

In [ ]:
data, labels = next(iter(load_train))
test_data, test_labels = next (iter(load_val))
data = data.view(data.shape[0], -1)
test_data = test_data.view(test_data.shape[0], -1)


In [ ]:
trainset = torchvision.datasets.MNIST(root=path_to_download + 'mnist/temp', train=True,
                                      download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.MNIST(root=path_to_download + 'mnist/temp', train=False,
                                     download=True, transform=transforms.ToTensor())
train_data=torch.Tensor(60000,28,28)
train_label=torch.LongTensor(60000)
for idx , example in enumerate(trainset):
  train_data[idx]=example[0].squeeze()
  train_label[idx]=example[1]
test_data=torch.Tensor(10000,28,28)
test_label=torch.LongTensor(10000)
for idx , example in enumerate(testset):
  test_data[idx]=example[0].squeeze()
  test_label[idx]=example[1]

Extracting /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw/train-images-idx3-ubyte.gz to /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw



Extracting /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw


Extracting /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw


Extracting /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/carlairismao/Documents/NTU masters 2020-21/AI6121 Computer Vision/Term Papermnist/temp/MNIST/raw
Processing...
Done!


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD( net1.parameters() , lr=lr )


In [ ]:
train_data.shape

torch.Size([60000, 28, 28])

In [ ]:
"""def get_error( score_s , label_s ):

    bs=score_s.size(0)
    predicted_labels = score_s.argmax(dim=1)
    indicator = (predicted_labels == label_s)
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs """

In [ ]:
def test_network():

    running_error=0
    num_batches=0

    for i in range(0,10000,bs):

        minibatch_data =  test_data[i:i+bs]
        minibatch_label= test_label[i:i+bs]

        inputs = minibatch_data.view(bs,784)

        scores=net1( inputs ) 

        error = get_error( scores , minibatch_label)

        running_error += error.item()

        num_batches+=1


    total_error = running_error/num_batches
    print( 'test error  = ', total_error*100 ,'percent')

In [ ]:
start = time.time()

for epoch in range(200):
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(60000)
 
    for count in range(0,60000,bs):
        
        # forward and backward pass
    
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices]
        minibatch_label= train_label[indices]

        inputs = minibatch_data.view(bs,784)

        inputs.requires_grad_()

        scores=net1( inputs ) 

        loss =  criterion( scores , minibatch_label) 
        
        loss.backward()

        optimizer.step()
        
        
        # compute some stats
        
        running_loss += loss.detach().item()
               
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1
    
    
    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats 
    # and compute the error rate on the test set  
    
    if epoch % 5 == 0 : 
      print(' ')

       
    
      print(' ')
        
      print('epoch=',epoch, '\t time=', elapsed_time,
            '\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
        
      test_network()
               

 
 
epoch= 0 	 time= 2.694605588912964 	 loss= 0.7058580060551564 	 error= 16.878333483139674 percent
test error  =  9.870000505447388 percent
 
 
epoch= 5 	 time= 15.839480876922607 	 loss= 0.28876011276617647 	 error= 8.141667272647222 percent
test error  =  7.950000488758087 percent
 
 
epoch= 10 	 time= 28.96443772315979 	 loss= 0.27220447459754843 	 error= 7.563333998123804 percent
test error  =  7.740000486373902 percent
 
 
epoch= 15 	 time= 42.1882483959198 	 loss= 0.26359951283937944 	 error= 7.33000066280365 percent
test error  =  7.550000500679016 percent
 
 
epoch= 20 	 time= 55.252307653427124 	 loss= 0.258605891947945 	 error= 7.136667400598526 percent
test error  =  7.6200004577636715 percent
 
 
epoch= 25 	 time= 68.46931004524231 	 loss= 0.25439077073149385 	 error= 7.035000689824422 percent
test error  =  7.530000483989715 percent
 
 
epoch= 30 	 time= 81.56419372558594 	 loss= 0.25204141123065105 	 error= 6.923334081967672 percent
test error  =  7.660000622272492 per

# Lenet5 CNN structure

In [2]:
# Set the device
device = torch.device("cuda")
# device= torch.device("cpu")
print("Device set: " + str(device))

Device set: cuda


In [3]:
# Utility methods

def get_error(scores, labels):
	bs = scores.size(0)
	predicted_labels = scores.argmax(dim=1)
	indicator = (predicted_labels == labels)
	num_matches = indicator.sum()
	return 1 - num_matches.float() / bs

In [4]:

# First eval on test set

def eval_on_test_set():
	running_error = 0
	num_batches = 0

	for i in range(0, 10000, bs):
		minibatch_data = test_data[i:i + bs].unsqueeze(dim=1)
		minibatch_label = test_label[i:i + bs]
		minibatch_data = minibatch_data.to(device)
		minibatch_label = minibatch_label.to(device)
		inputs = (minibatch_data - mean) / std
		scores = net(inputs)
		error = get_error(scores, minibatch_label)
		running_error += error.item()
		num_batches += 1
	total_error = running_error / num_batches
	print('error rate on test set =', total_error * 100, 'percent')

In [5]:
# Download MNIST dataset form pytorch
print("Checking for MNIST dataset.. ")


def check_mnist_dataset_exists(path_data='./data/'):
	import os.path
	flag_train_data = os.path.isfile(path_data + 'mnist/train_data.pt')
	flag_train_label = os.path.isfile(path_data + 'mnist/train_label.pt')
	flag_test_data = os.path.isfile(path_data + 'mnist/test_data.pt')
	flag_test_label = os.path.isfile(path_data + 'mnist/test_label.pt')
	if flag_train_data == False or flag_train_label == False or flag_test_data == False or flag_test_label == False:
		print('MNIST dataset missing - downloading...')
		import torchvision
		import torchvision.transforms as transforms
		trainset = torchvision.datasets.MNIST(root=path_data + 'mnist/temp', train=True,
		                                      download=True, transform=transforms.ToTensor())
		testset = torchvision.datasets.MNIST(root=path_data + 'mnist/temp', train=False,
		                                     download=True, transform=transforms.ToTensor())
		train_data = torch.Tensor(60000, 28, 28)
		train_label = torch.LongTensor(60000)
		for idx, example in enumerate(trainset):
			train_data[idx] = example[0].squeeze()
			train_label[idx] = example[1]
		torch.save(train_data, path_data + 'mnist/train_data.pt')
		torch.save(train_label, path_data + 'mnist/train_label.pt')
		test_data = torch.Tensor(10000, 28, 28)
		test_label = torch.LongTensor(10000)
		for idx, example in enumerate(testset):
			test_data[idx] = example[0].squeeze()
			test_label[idx] = example[1]
		torch.save(test_data, path_data + 'mnist/test_data.pt')
		torch.save(test_label, path_data + 'mnist/test_label.pt')
	train_data = torch.load(path_data + 'mnist/train_data.pt')
	train_label = torch.load(path_data + 'mnist/train_label.pt')
	test_data = torch.load(path_data + 'mnist/test_data.pt')
	test_label = torch.load(path_data + 'mnist/test_label.pt')
	return train_data, train_label, test_data, test_label




Checking for MNIST dataset.. 


In [6]:
train_data, train_label, test_data, test_label = check_mnist_dataset_exists()

print("Creating LeNet")


MNIST dataset missing - downloading...


Extracting ./data/mnist/temp/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/temp/MNIST/raw


Extracting ./data/mnist/temp/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/temp/MNIST/raw


Extracting ./data/mnist/temp/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/temp/MNIST/raw


Extracting ./data/mnist/temp/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/temp/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Creating LeNet


In [7]:

class LeNet(nn.Module):

	def __init__(self):
		super(LeNet, self).__init__()

		# CL1:   28 x 28  -->    50 x 28 x 28
		self.conv1 = nn.Conv2d(1, 50, kernel_size=3, padding=1)
		# MP1: 50 x 28 x 28 -->    50 x 14 x 14
		self.pool1 = nn.MaxPool2d(2, 2)
		# CL2:   50 x 14 x 14  -->    100 x 14 x 14
		self.conv2 = nn.Conv2d(50, 100, kernel_size=3, padding=1)
		# MP2: 100 x 14 x 14 -->    100 x 7 x 7
		self.pool2 = nn.MaxPool2d(2, 2)
		# LL1:   100 x 7 x 7 = 4900 -->  100
		self.linear1 = nn.Linear(4900, 100)
		# LL2:   100  -->  10
		self.linear2 = nn.Linear(100, 10)

In [8]:
	def forward(self, x):
		# CL1:   28 x 28  -->    50 x 28 x 28
		x = self.conv1(x)
		x = F.relu(x)
		# MP1: 50 x 28 x 28 -->    50 x 14 x 14
		x = self.pool1(x)
		# CL2:   50 x 14 x 14  -->    100 x 14 x 14
		x = self.conv2(x)
		x = F.relu(x)
		# MP2: 100 x 14 x 14 -->    100 x 7 x 7
		x = self.pool2(x)
		# LL1:   100 x 7 x 7 = 4900  -->  100
		x = x.view(-1, 4900)
		x = self.linear1(x)
		x = F.relu(x)
		# LL2:   4900  -->  10
		x = self.linear2(x)

		return x


In [ ]:
net = LeNet()
print(net)
net = net.to(device)

In [ ]:
# Set hyper parameters
# 1. loss function
# 2. learning rate
# 3. batch size

criterion = nn.CrossEntropyLoss()
lr = 0.25
bs = 128
mean = train_data.mean()
std = train_data.std()

In [10]:
start = time.time()

for epoch in range(1, 30):

	if not epoch % 5:
		lr = lr / 1.5

	optimizer = torch.optim.SGD(net.parameters(), lr=lr)

	running_loss = 0
	running_error = 0
	num_batches = 0

	shuffled_indices = torch.randperm(60000)

	for count in range(0, 60000, bs):
		# FORWARD AND BACKWARD PASS
		optimizer.zero_grad()
		indices = shuffled_indices[count:count + bs]
		minibatch_data = train_data[indices].unsqueeze(dim=1)
		minibatch_label = train_label[indices]
		minibatch_data = minibatch_data.to(device)
		minibatch_label = minibatch_label.to(device)
		inputs = (minibatch_data - mean) / std  # ONLY CHANGE IS HERE!
		inputs.requires_grad_()
		scores = net(inputs)
		loss = criterion(scores, minibatch_label)
		loss.backward()
		optimizer.step()
		# COMPUTE STATS
		running_loss += loss.detach().item()
		error = get_error(scores.detach(), minibatch_label)
		running_error += error.item()
		num_batches += 1

	# AVERAGE STATS THEN DISPLAY
	total_loss = running_loss / num_batches
	total_error = running_error / num_batches
	elapsed = (time.time() - start) / 60

	print('epoch=', epoch, '\t time=', elapsed, 'min', '\t lr=', lr,
	      '\t loss=', total_loss, '\t error=', total_error * 100, 'percent')

	eval_on_test_set()

NameError: ignored